In [1]:
import metpy.calc as mpcalc
from metpy.units import units
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature,NaturalEarthFeature
import matplotlib as mpl
import numpy.ma as ma


import metpy 
from metpy.plots import USCOUNTIES # Make sure metpy is updated to latest version.

%matplotlib inline
plt.rcParams.update({"font.size":30})
mpl.rcParams['legend.fontsize'] = 'large'

In [2]:
from pathlib import Path

HRRR= Path("/export/home/mbrewer/HRRR/hrrr.t18z.wrfprsf00_1108.nc")

el = Path("/export/home/mbrewer/Documents/GMTED2010_15n030_0125deg.nc")

In [3]:
ds = xr.open_dataset(HRRR)
elev = xr.open_dataset(el)

data = ds.metpy.parse_cf()

In [4]:
wp_lat =  39.697254
wp_lon = -121.574221

lat = data.gridlat_0
lon = data.gridlon_0
               
abslat = np.abs(lat-wp_lat)
abslon= np.abs(lon-wp_lon)
c = np.maximum(abslon,abslat)
xx, yy = np.where(c == np.min(c))
print(xx,yy) 

[663] [220]


In [23]:
height = ds.HGT_P0_L100_GLC0
uwind_pres = ds.UGRD_P0_L100_GLC0[:,0:int(yy+1000),0:int(xx+500)]
vwind_pres =ds.VGRD_P0_L100_GLC0[:,0:int(yy+1000),0:int(xx+500)]
uwind_10m = ds.UGRD_P0_L103_GLC0[0,0:int(yy+1000),0:int(xx+500)]
vwind_10m = ds.VGRD_P0_L103_GLC0[0,0:int(yy+1000),0:int(xx+500)]
RH = ds.RH_P0_L100_GLC0[:,0:int(yy+1000),0:int(xx+500)]
RH_2m =ds.RH_P0_L103_GLC0[0:int(yy+1000),0:int(xx+500)]
DPT = ds.DPT_P0_L103_GLC0[0:int(yy+1000),0:int(xx+500)]
#lon_2d, lat_2d = np.meshgrid(ds['gridlon_0'], ds['gridlat_0'])

In [6]:
# Function used to create the map subplots
def plot_background(ax):
    ax.set_extent([-108+360.,-126.5+360., 32., 49.])
    ax.coastlines(resolution='10m', linewidth=2, color = 'black', zorder = 4)
    political_boundaries = NaturalEarthFeature(category='cultural',
                                   name='admin_0_boundary_lines_land',
                                   scale='10m', facecolor='none')
    states = NaturalEarthFeature(category='cultural',
                                   name='admin_1_states_provinces_lines',
                                   scale='50m', facecolor='none')

    ax.add_feature(political_boundaries, linestyle='-', edgecolor='black', zorder =4)
    ax.add_feature(states, linestyle='-', edgecolor='black',linewidth=2, zorder =4)
    #gl = ax.gridlines(draw_labels=True)
    #gl.xlabels_top = gl.ylabels_right = False
    #gl.xformatter = LONGITUDE_FORMATTER
    #gl.yformatter = LATITUDE_FORMATTER
    return ax

In [8]:
RH = data.metpy.parse_cf('RH_P0_L100_GLC0')
RH = RH[:,0:int(yy+1000),0:int(xx+500)]
x, y = RH.metpy.coordinates('x', 'y')


In [7]:
lccProjParams_HRRR = { 'central_latitude'   : -38.5,        # same as lat_0 in proj4 string 
                       'central_longitude'  : -97.5,        # same as lon_0
                       'standard_parallels' : (38.5, 38.5), # same as (lat_1, lat_2)
                     }

crs = ccrs.LambertConformal(**lccProjParams_HRRR)

In [ ]:
var = RH

fig, ax = plt.subplots(figsize = (20,20),subplot_kw={'projection': crs})
plot_background(ax)
clevs = np.arange(0.,105.,1)
levs = np.arange(0,6000.,500)
levs2 = np.arange(1,3000.,250)
cf = ax.contourf(x,y,var[26], clevs, transform = ccrs.PlateCarree(), cmap = 'viridis_r', alpha = .7, zorder = 2, vmax = 80)
cs1 =ax.contour(elev.longitude,elev.latitude, elev.elevation, levs, transform = ccrs.PlateCarree(), colors = '#333333', zorder = 1)
#cs =ax.contour(lon_2d,lat_2d, heights_700,levs, transform = ccrs.PlateCarree(), linewidths = 3, colors = '#116000')
#ax.clabel(cs, cs.levels, fontsize=20, colors='k')

ax.scatter(-121.6219, 39.7596, s =300,  marker = '*', label = 'Paradise, California', transform = ccrs.PlateCarree(), color = 'tab:red', zorder =6)
sknum = 15
skip=(slice(None,None,sknum),slice(None,None,sknum))
ax.barbs(x[skip].values,y[skip].values, uwind_pres[26,0:int(yy+1000),0:int(xx+600)][skip].values, vwind_pres[26,0:int(yy+1000),0:int(xx+600)][skip].values, length=6,
             sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
             zorder = 5,
             linewidth=0.95, transform= ccrs.PlateCarree())
#ax.barbs(x[::50].values,y[::50].values, uwind_pres[26][::50].values, vwind_pres[26][::50].values, transform = ccrs.PlateCarree(), zorder = 5)
#ax.set_title('201]-11-08 0000UTC GFS 0.5°', fontsize = 30)
ax.add_feature(USCOUNTIES.with_scale('500k'), edgecolor='black', linewidth=.2, zorder = 4)
ax.legend(loc = 1,fontsize = '18')
cb = fig.colorbar(cf, shrink=0.74, pad=0)
font_size = 20 # Adjust as appropriate.
cb.ax.tick_params(labelsize=font_size)
cb.set_label('RH (%)', size = 'x-large', fontsize = 22 )
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontname('Arial')
    label.set_fontsize(30)
    label.set_fontweight('bold');
plt.title('HRRR Model Output', loc='left', fontweight='bold', fontsize = 22)
plt.title('Field: %s' % (var.attrs['long_name']), loc='center', fontsize = 18)
plt.title('Valid Time: %s\nLevel: %s hPa' % (var.attrs['initial_time'], int(var[26].coords['lv_ISBL0'].data)/100), loc='right', fontsize = 18)
plt.savefig('HRRR_RH_%s_%sz_%s.png'% (var.attrs['initial_time'][3:5],var.attrs['initial_time'][12:14], int(var[26].coords['lv_ISBL0'].data)/100), dpi = 800, bbox_inches = 'tight')